In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

### Conv2D 적용하기
* Conv2D() 를 모델에 적용 시에는 반드시 입력은 배치 크기를 제외하고 3차원이 되어야 함(즉 배치를 포함하면 4차원)  

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model

# RGB: (28, 28, 3) / greyscale: (28, 28, 1)
input_tensor = Input(shape=(28, 28, 1)) # cf. (28, 28) -> (28, 28, 1) 이때 1은 채널의 차원
# Conv: 배치를 제외해. 무조건 3차원을 받음. 받고 나서 배치(4차원) 
x = Conv2D(filters=4, kernel_size=3, strides=1, padding='same', activation='relu')(input_tensor) # output: feature map(배치를 제외한 3차원)
print('x type:', type(x), 'x:', x) # 피처맵: 3차원!! 배치 사이즈 제외!!
# 필터: 커널이 여러 개가 모임 -> 커널은 2차원 -> 1개의 필터는 무조건 3차원 (channels = kernels) 
# filters=4: shape=(None, 28, 28, 4) -> 필터 1개는 무조건 3차원... 피처맵도 3차원... -> 여기 이해가 가장 어렵

In [ ]:
# input_grey = Input(shape=(28, 28, 1)) # greyscale
# input_rgb = Input(shape=(28, 28, 3)) # RGB

In [ ]:
# x = Conv2D(filters=4, 
#            kernel_size=3, 
#            strides=1, 
#            padding='same', 
#            activation='relu') 
#            (input_tensor) # output: feature map(배치를 제외한 3차원 형태)

### Pooling 적용하기

In [ ]:
input_tensor = Input(shape=(28, 28, 1)) 
x = Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu')(input_tensor)
x = MaxPooling2D(2)(x)
print(x)
# filters=16: shape=(None, 14, 14, 16)

### CNN 모델 생성

In [ ]:
input_tensor = Input(shape=(28, 28, 1)) # (None, 28, 28, 1)
x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(input_tensor) # (None, 28, 28, 32)
x = Conv2D(filters=64, kernel_size=3, activation='relu')(x) # (None, 26, 26, 64)
x = MaxPooling2D(2)(x) # # (None, 13, 13, 64)

model = Model(inputs=input_tensor, outputs=x)
model.summary()

### 중요
# 하나의 필터/피처맵은 무조건 3차원이다.
# Conv에서 말하는 3차원은 배치를 제외한 3차원이다.
# 여러 개의 커널 = 필터 -> 필터의 채널(커널) 개수는 keras가 자동적으로 맞춰줌(?) 
# Params: 3*3*32*64+64 = 18496 (=weight의 개수)
# 3*3*32: 하나의 필터(3차원)
# *64: 필터의 개수
# +64: bias

In [ ]:
from tensorflow.keras.layers import Dense, Flatten

input_tensor = Input(shape=(28, 28, 1))
x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(input_tensor)
x = Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = MaxPooling2D(2)(x) # pooling은 이미 나온 피처맵을 줄이는 거니까 학습할 파라미터는 없음 

# 3차원으로 되어있는 Feature map 결과를 Fully Connected 연결하기 위해서는 Flatten()을 적용해야함. 
# Flatten(): 3차원 -> 1차원으로 변환하는 작업 
x = Flatten()(x)
x = Dense(100, activation='relu')(x) # 바로 붙이면 손실이 많을 수 있으니 Dense layer 추가 -> Dense를 하는 순간 파라미터 급 증가 -> 오버피팅 위험 -> dropout
output = Dense(10, activation='softmax')(x) # 최종 classification을 위한 softmax
model = Model(inputs=input_tensor, outputs=output)
model.summary()


In [ ]:
input_tensor = Input(shape=(28, 28, 1))
x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(input_tensor) 
x = Conv2D(filters=64, kernel_size=3, activation='relu')(x) 
x = MaxPooling2D(2)(x) 
x = Flatten()(x) 
x = Dense(100, activation='relu')(x) 
x = Dropout(0.4)(x) 
output = Dense(10, activation='softmax')(x)

model = Model(inputs=input_tensor, outputs=output)
model.summary()

In [ ]:
# input_tensor = Input(shape=(28, 28, 1)) # (None, 28, 28, 1)
# x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(input_tensor) # (None, 28, 28, 32)
# x = Conv2D(filters=64, kernel_size=3, activation='relu')(x) # (None, 26, 26, 64)
# x = MaxPooling2D(2)(x) # # (None, 13, 13, 64) cf. pooling은 이미 나온 피처맵을 줄이는 거니까 학습할 파라미터는 없음
# x = Flatten()(x) # Flatten(): 3차원 피처맵 -> 1차원 FC로 연결
# x = Dense(100, activation='relu')(x) # 바로 붙이면 손실이 많을 수 있으니 Dense layer 추가 
# x = Dropout(0.4)(x) # Dense를 추가하는 순간 파라미터 급 증가 -> 오버피팅 위험 -> dropout
# output = Dense(10, activation='softmax')(x) # 최종 classification을 위한 softmax 연결

# model = Model(inputs=input_tensor, outputs=output)
# model.summary()

### 중요
# 하나의 필터/피처맵은 무조건 3차원이다.
# Conv에서 말하는 3차원은 배치를 제외한 3차원이다.
# 여러 개의 커널 = 필터 -> 필터의 채널(커널) 개수는 keras가 자동적으로 맞춰준다. 
# Params: 3*3*32*64+64 = 18496 (=weight의 개수)
# 3*3*32: 하나의 필터(3차원)
# *64: 필터의 개수
# +64: bias

### Fashion MNIST 데이터 전처리후 모델 학습

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# 전체 6만개 데이터 중, 5만개는 학습 데이터용, 1만개는 테스트 데이터용으로 분리
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

def get_preprocessed_data(images, labels):
    
    # 학습과 테스트 이미지 array를 0~1 사이값으로 scale 및 float32 형 변형. 
    images = np.array(images/255.0, dtype=np.float32)
    labels = np.array(labels, dtype=np.float32)
    
    return images, labels

# 0 ~ 1사이값 float32로 변경하는 함수 호출 한 뒤 OHE 적용 
def get_preprocessed_ohe(images, labels):
    images, labels = get_preprocessed_data(images, labels)
    # OHE 적용 
    oh_labels = to_categorical(labels)
    return images, oh_labels

# 학습/검증/테스트 데이터 세트에 전처리 및 OHE 적용한 뒤 반환 
def get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021):
    # 학습 및 테스트 데이터 세트를  0 ~ 1사이값 float32로 변경 및 OHE 적용. 
    train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
    test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)
    
    # 학습 데이터를 검증 데이터 세트로 다시 분리
    tr_images, val_images, tr_oh_labels, val_oh_labels = train_test_split(train_images, train_oh_labels, test_size=valid_size, random_state=random_state)
    
    return (tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels ) 


# Fashion MNIST 데이터 재 로딩 및 전처리 적용하여 학습/검증/데이터 세트 생성. 

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)
(tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels) = \
    get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021)
print(tr_images.shape, tr_oh_labels.shape, val_images.shape, val_oh_labels.shape, test_images.shape, test_labels.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy

model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x=tr_images, y=tr_oh_labels, batch_size=128, epochs=30, validation_data=(val_images, val_oh_labels))

### 모델 성능 평가

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_history(history):
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='valid')
    plt.legend()
    
show_history(history)

In [ ]:
# 테스트 데이터 세트로 모델 성능 검증
model.evaluate(test_images, test_oh_labels, batch_size=256, verbose=1)

### Dropout을 적용하여 Fully Connected Layer의 오버피팅 조정
* CNN은 일반적으로 Dense Layer보다는 파라미터수(weight 수) 작음
* 하지만 많은 Filter 들을 적용하고 이를  Fully Connected Layer로 연결 시 파라미터 수가 늘어남. 
* Flatten() 이후 Dropout을 적용하여 특정 비율로 FC Layer 연결을 누락 적용. 

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout

input_tensor = Input(shape=(28, 28, 1))
x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(input_tensor)
x = Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = MaxPooling2D(2)(x)

x = Flatten()(x) # Flatten -> Dense를 FC로 연결하면 overfitting의 위험이 있음 
x = Dropout(rate=0.5)(x) # 일반적인 것에도 확장되는 모델을 만들기 위해 Dropout layer 추가
x = Dense(100, activation='relu')(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=input_tensor, outputs=output)
model.summary()

In [ ]:
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x=tr_images, y=tr_oh_labels, batch_size=128, epochs=30, validation_data=(val_images, val_oh_labels))

In [ ]:
show_history(history)
model.evaluate(test_images, test_oh_labels, batch_size=256, verbose=1)

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D 

def create_model():
    input_tensor = Input(shape=(28, 28, 1))
    x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(input_tensor)
    x = Conv2D(filters=64, kernel_size=3, activation='relu')(x)
    x = MaxPooling2D(2)(x)

    #x = Dropout(rate=0.5)(x) -> 조금 더 적극적인 dropout 모델
    x = Flatten()(x) # 요즘은 flatten 안하는 추세 -> GlobalAveragePooling2D로 대체되는 중?
    x = Dropout(rate=0.5)(x)
    x = Dense(200, activation='relu')(x)
    X = Dropout(rate=0.2)(x)
    output = Dense(10, activation='softmax')(x)
    model = Model(inputs=input_tensor, outputs=output)
    model.summary()
    
    return model

model = create_model()

In [ ]:
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x=tr_images, y=tr_oh_labels, batch_size=128, epochs=30, validation_data=(val_images, val_oh_labels))

In [ ]:
show_history(history)
model.evaluate(test_images, test_oh_labels, batch_size=256, verbose=1)

### 입력 이미지는 배치를 포함하여 4차원이 되어야 함(즉 배치를 제외하면 3차원)
* Conv2D()는 입력으로 배치를 제외하고 3차원 입력이 되어야 함. 
* 하지만 2차원으로 입력해도 Input(shape=(28, 28, 1)) 에서 3차원으로 변경함. 
* 명확하게는 2차원 Grayscale이미지더라도 입력 numpy 이미지 배열에서 배치를 제외한 3차원 입력을 만들어 주는게 좋음. 

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

print('before reshape:', train_images.shape, test_images.shape)
train_images = np.reshape(train_images, (60000, 28, 28, 1))
test_images = np.reshape(test_images, (10000, 28, 28, 1))
print('after reshape:', train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels) = \
    get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021)
print(tr_images.shape, tr_oh_labels.shape, val_images.shape, val_oh_labels.shape, test_images.shape, test_labels.shape)

In [ ]:
from tensorflow.keras.optimizers import Adam

model = create_model()
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x=tr_images, y=tr_oh_labels, batch_size=128, epochs=30, validation_data=(val_images, val_oh_labels))

In [ ]:
show_history(history)
model.evaluate(test_images, test_oh_labels, batch_size=256, verbose=1)

### Stride가 1이고 Padding이 없는 경우
* I는 입력 Feature Map의 크기, F는 Filter의 크기(Kernel size), P는 Padding(정수), S는 Strides(정수)
* O = (I - F + 2P)/2 + 1 = (5 - 3 + 0 )/1 + 1 = 3

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model

input_tensor = Input(shape=(5, 5, 1))
x = Conv2D(filters=1, kernel_size=3, strides=1)(input_tensor)
print('x.shape:', x.shape)

### Stride가 1이고 Padding이 1인 경우
* O = (I - F + 2P)/2 + 1 = (5 - 3 + 2 )/1 + 1 = 5

In [ ]:
input_tensor = Input(shape=(5, 5, 1))
x = Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(input_tensor)
print('x.shape:', x.shape)

In [ ]:
# ZeroPadding2D Layer를 이용하여 padding을 수동으로 적용. 
from tensorflow.keras.layers import ZeroPadding2D

input_tensor = Input(shape=(5, 5, 1))
padded_input = ZeroPadding2D(padding=1)(input_tensor)
print('shape after padding:', padded_input.shape)
x = Conv2D(filters=1, kernel_size=3, strides=1)(padded_input)
print('x.shape:', x.shape)


### Stride가 2이고 Padding이 없는 경우 
* O = (I - F + 2P)/2 + 1 = (5 - 3)/2 + 1 = 2

In [ ]:
input_tensor = Input(shape=(5, 5, 1))
x = Conv2D(filters=1, kernel_size=3, strides=2)(input_tensor)
print('x.shape:', x.shape)

### Stride가 2이고 Padding은 1 적용
* O = (I - F + 2P)/2 + 1 = (5 - 3 + 2)/2 + 1 = 3

In [ ]:
input_tensor = Input(shape=(5, 5, 1))
padded_input = ZeroPadding2D(padding=1)(input_tensor)
print('shape after padding:', padded_input.shape)
x = Conv2D(filters=1, kernel_size=3, strides=2)(padded_input)
print('x.shape:', x.shape)

### 입력이 6X6에서 Stride가 2 적용
* O = (I - F + 2P)/2 + 1 = (6 - 3 + 0)/2 + 1 = 2.5 = 2

In [ ]:
input_tensor = Input(shape=(6, 6, 1))
x = Conv2D(filters=1, kernel_size=3, strides=2)(input_tensor)
print('x.shape:', x.shape)

In [ ]:
input_tensor = Input(shape=(6, 6, 1))
x = Conv2D(filters=1, kernel_size=3, strides=2, padding='same')(input_tensor)
print('x.shape:', x.shape)

In [ ]:
input_tensor = Input(shape=(6, 6, 1))
padded_input = ZeroPadding2D(padding=1)(input_tensor)
x = Conv2D(filters=1, kernel_size=3, strides=2, padding='valid')(padded_input)
print('x.shape:', x.shape)

In [ ]:
input_tensor = Input(shape=(6, 6, 1))
padded_input = ZeroPadding2D(padding=((1, 0),(1,0)))(input_tensor) # padding = 튜플: 위&왼쪽만 패딩
x = Conv2D(filters=1, kernel_size=3, strides=2)(padded_input)
print('x.shape:', x.shape)

### Maxpooling 적용

In [ ]:
input_tensor = Input(shape=(223, 223, 1))
x = MaxPooling2D(2)(input_tensor)
print('x.shape:', x.shape)